In [1]:
import os
import json
import pandas as pd
import traceback


In [2]:
from langchain.chat_models import ChatOpenAI


In [4]:
from dotenv import load_dotenv

load_dotenv()

False

In [ ]:
KEY=os.getenv("OPENAI_API_KEY")


In [3]:
llm=ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo",temperature=0.5)



ValidationError: 1 validation error for ChatOpenAI
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. (type=value_error)

In [ ]:
llm

In [ ]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [1]:
RESPONSE_JSON={
    "1":{
        "mcq":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer",
    },
    "2":{
        "mcq":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer",
    },
"3":{
        "mcq":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer",
    },

}

NameError: name 'RESPONSE' is not defined

In [ ]:
Template="""
Text:{text}
You are an expert MCQ maker.Given the above text,it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be confirming the text as well.
Make sure to format the response like RESPONSE_JSON below and use it as a guide.\
Ensure to make {number} MCQS
### RESPONSE_JSON
{response_json}
 """

In [ ]:
quiz_generation_prompt=PromptTemplate(input_variables=["text","number","subject","tone","response_json"],template=Template)


In [ ]:
quiz_chain=LLMChain(llm=llm,prompt=quiz_generation_prompt,output_key="quiz",verbose=True)

In [ ]:
TEMPLATE2="""You are an expert english grammarin and writer.Given a Multiple Choice Quiz for {subject} students.\
    You need ot evaluate the complexity of the question and give a complete analysis of the quiz.Only use at max 50 words for complexity
    if the quiz is not at par with the cognittive and analytical abilities of the students,\
        update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
        Quiz _MCQs:
        {quiz}
        
        check from an expert English Writer of the above quiz
        """

In [ ]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject","quiz"],template=Template)

In [ ]:
review_chain=LLMChain(llm=llm,prompt=quiz_evaluation_prompt,output_key="review",verbose=True)


In [ ]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain,review_chain],input_variables=["text","number","subject","tone","response_json","quiz"],output_variables=["quiz","review"],vebose=True)

In [ ]:

file_path=""

In [ ]:
with open(file_path,'r') as file:
    TEXT=file.read()

In [2]:
#serialize the python dictionary into a json formatted string
json.dump(RESPONSE_JSON)

NameError: name 'json' is not defined

In [ ]:
#How to setup token usage tracking in langchain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text":TEXT,
            "number":NUMBER,
            "subject":SUBJECT,
            "tone":TONE,
            "response_json":json.dumps(RESPONSE_JSON)

        }
    )

In [ ]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total cost:{total_cost}")

In [ ]:
quiz=response.get("quiz")

In [ ]:
json.load(quiz)

In [ ]:
quiz_table_data=[]
for key,value in quiz.items():
    mcq=value["mcq"]
    options=value["mcq"]
    options=" | ".join(
        [
            f"{option}:{option_value}"
            for option,option_value in value["options"].items()
        ]
    )
    correct=value["correct"]
    quiz_table_data.append({"MCQ":mcq,"choices":options,"Correct":correct})

In [ ]:
quiz=pd.DataFrame(quiz_table_data)

In [ ]:
quiz.to_csv("machinelearning.csv",index=False)